# Dowload github and the necessary libraries to train the model

In [1]:
!git clone https://github.com/GLOMQuyet/symato.git
%cd /content/symato
!pip install -r requirements.txt

Cloning into 'symato'...
remote: Enumerating objects: 1145, done.
remote: Counting objects: 100% (577/577), done.
remote: Compressing objects: 100% (277/277), done.
remote: Total 1145 (delta 345), reused 500 (delta 296), pack-reused 568
Receiving objects: 100% (1145/1145), 19.30 MiB | 14.41 MiB/s, done.
Resolving deltas: 100% (644/644), done.
/content/symato
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Dowload and unzip dataset public from discord

In [2]:
%cd rwkv-v4neo
!wget https://cdn.discordapp.com/attachments/1070553582111297616/1071335804451954688/vlc.zip
!unzip vlc.zip

/content/symato/rwkv-v4neo
--2023-02-20 13:12:23--  https://cdn.discordapp.com/attachments/1070553582111297616/1071335804451954688/vlc.zip
Resolving cdn.discordapp.com (cdn.discordapp.com)... 162.159.130.233, 162.159.134.233, 162.159.133.233, ...
Connecting to cdn.discordapp.com (cdn.discordapp.com)|162.159.130.233|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4738750 (4.5M) [application/zip]
Saving to: ‘vlc.zip’

vlc.zip             100%[===================>]   4.52M  3.50MB/s    in 1.3s    

2023-02-20 13:12:25 (3.50 MB/s) - ‘vlc.zip’ saved [4738750/4738750]

Archive:  vlc.zip
  inflating: vlc.txt                 
  inflating: vlc.xyz                 


# Train Tiny model

In [3]:
!python3 train.py --load_model "" --wandb "" --proj_dir "out" \
--data_file "vlc.xyz" --data_type "symato" --vocab_size 0 \
--ctx_len 512 --epoch_steps 2000 --epoch_count 20 --epoch_begin 0 --epoch_save 5 \
--micro_bsz 128 --n_layer 4 --n_embd 320 --pre_ffn 0 --head_qk 0 \
--lr_init 8e-4 --lr_final 1e-6 --warmup_steps 0 --beta1 0.9 --beta2 0.99 \
--accelerator gpu --devices 1 --precision fp16 --strategy ddp_find_unused_parameters_false --grad_cp 0

2023-02-20 13:12:33.000006: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-20 13:12:34.195567: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-20 13:12:34.195686: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-20 13:12:34.195708: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

#Train Base model

In [6]:
!python3 train.py --data_order=reversed --load_model "" --wandb "" --proj_dir "out" \
--data_file "vlc.xyz" --data_type "symato" \
--ctx_len 640 --epoch_steps 2000 --epoch_count 20 --epoch_begin 0 --epoch_save 5 \
--micro_bsz 64 --n_layer 6 --n_embd 512 --pre_ffn 0 --head_qk 0 \
--lr_init 8e-4 --lr_final 1e-6 --warmup_steps 0 --beta1 0.9 --beta2 0.99 \
--accelerator gpu --devices 1 --precision fp16 --strategy ddp_find_unused_parameters_false

2023-02-20 13:15:49.503252: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-20 13:15:50.395912: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-20 13:15:50.396030: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-20 13:15:50.396052: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

# Train with tokenizer byte2byte from Phobert

In [18]:
!mkdir 'data'

In [19]:
%cd data
!wget https://huggingface.co/vinai/phobert-base/raw/main/vocab.txt;
!wget https://huggingface.co/vinai/phobert-base/raw/main/tokenizer.json;
!wget https://huggingface.co/vinai/phobert-base/raw/main/bpe.codes;
%cd /content/symato/rwkv-v4neo
!python3 train.py --load_model "" --wandb "pho-vlc" --proj_dir "out" \
--data_file "vlc.txt" --data_type "utf-8" \
--ctx_len 192 --epoch_steps 2000 --epoch_count 20 --epoch_begin 0 --epoch_save 5 \
--micro_bsz 64 --n_layer 5 --n_embd 320 --pre_ffn 0 --head_qk 0 \
--lr_init 8e-4 --lr_final 1e-5 --warmup_steps 0 --beta1 0.9 --beta2 0.99\
--accelerator gpu --devices 1 --precision fp16 --strategy ddp_find_unused_parameters_false --grad_cp 0

/content/symato/rwkv-v4neo/data
--2023-02-20 13:24:31--  https://huggingface.co/vinai/phobert-base/raw/main/vocab.txt
Resolving huggingface.co (huggingface.co)... 3.231.67.228, 54.235.118.239, 2600:1f18:147f:e850:e203:c458:10cd:fc3c, ...
Connecting to huggingface.co (huggingface.co)|3.231.67.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 895321 (874K) [text/plain]
Saving to: ‘vocab.txt’

vocab.txt           100%[===================>] 874.34K   790KB/s    in 1.1s    

2023-02-20 13:24:33 (790 KB/s) - ‘vocab.txt’ saved [895321/895321]

--2023-02-20 13:24:33--  https://huggingface.co/vinai/phobert-base/raw/main/tokenizer.json
Resolving huggingface.co (huggingface.co)... 3.231.67.228, 54.235.118.239, 2600:1f18:147f:e850:e203:c458:10cd:fc3c, ...
Connecting to huggingface.co (huggingface.co)|3.231.67.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3132320 (3.0M) [text/plain]
Saving to: ‘tokenizer.json’

tokenizer.json      100%[===